In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tqdm
%matplotlib inline

In [ ]:
class Undistorter(object):
    
    def __init__(self, nx=7, ny=9):
        self.nx = nx
        self.ny = ny
        
        self.singleObjP = np.zeros((nx*ny, 3), np.float32)
        self.singleObjP[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)
        
    def clearStorage(self):
        self.calibrationErrors = []
        self.imgp = []
        
    def fitImg(self, img):
        if isinstance(img, str):
            img = cv2.imread(img)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        self.imageShape = gray.shape[::-1]
        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        ret, corners = cv2.findChessboardCorners(gray, (self.nx, self.ny), None)
        if ret:
            self.imgp.append(corners)
            
    def fit(self, imgs, nerr=0, earlyStoppingRatio=None, minImages=20):
        self.clearStorage()
        from sklearn.utils import shuffle
        for img in tqdm.tqdm_notebook(shuffle(imgs), unit='frame'):
            self.fitImg(img)
            if earlyStoppingRatio is not None and len(self.imgp) > 0:
                self.calcParams()
            if len(self.calibrationErrors) > minImages and earlyStoppingRatio is not None:
                ratio = self.calibrationErrors[-1] / min(self.calibrationErrors)
                print(ratio)
                # Aribtrary threshold on rising calibration error.
                if ratio > earlyStoppingRatio:
                    print('Early termination due to potential overfitting.')
                    break
        if earlyStoppingRatio is None:
            self.calcParams()
        
    def calcParams(self):
        objp = [self.singleObjP] * len(self.imgp)
        ret, self.mtx, self.dist, self.rvecs, self.tvecs = cv2.calibrateCamera(
            objp, self.imgp, self.imageShape, None, None
        )
        self.calibrationErrors.append(ret)
        return ret
    
    def optimalMatrix(self, img, alpha=1):
        h,  w = img.shape[:2]
        newcameramtx, roi=cv2.getOptimalNewCameraMatrix(
            self.mtx, self.dist, (w,h), alpha, (w,h)
        )
        return newcameramtx, roi
    
    def remap(self, img, optimalMatrix=True, cropRoi=False, m1type=cv2.CV_32FC1):
        if isinstance(img, str):
            img = cv2.imread(img)
        
        h, w = img.shape[:2]
        if optimalMatrix:
            newcameramtx, roi = self.optimalMatrix(img)
        else:
            newcameramtx = self.mtx
            
        mapx, mapy = cv2.initUndistortRectifyMap(
            self.mtx, dist, np.array([]), newcameramtx, (w, h), m1type
        )
        
        dst = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)
        
        if cropRoi:
            x, y, w, h = roi
            dst = dst[y:y+h, x:x+w]
        return dst
        
                
    def __call__(self, img, optimalMatrix=True, cropRoi=False, alpha=1):
        if isinstance(img, str):
            img = cv2.imread(img)
            
        if optimalMatrix:
            newcameramtx, roi = self.optimalMatrix(img, alpha=alpha)
        else:
            newcameramtx = self.mtx
        dst = cv2.undistort(img, self.mtx, self.dist, None, newcameramtx)
        
        if cropRoi:
            x, y, w, h = roi
            dst = dst[y:y+h, x:x+w]
        return dst
    
    def reprojectionErrorPlot(self, **plotKwargs):
        fig, ax = plt.subplots()
        ax.plot(self.calibrationErrors, **plotKwargs)
        ax.set_xlabel('number of images')
        ax.set_ylabel('calibration RMS re-projection error ');
        return fig, ax

In [ ]:
import glob
imgPaths = glob.glob('camera_cal/calibration*.jpg')

In [ ]:
undistorter = Undistorter(nx=9, ny=6)
undistorter.fit(imgPaths)

In [ ]:
for imgPath in tqdm.tqdm_notebook(imgPaths[3:5]):
    dist = cv2.imread(imgPath)
    undist = undistorter(imgPath, cropRoi=True)
    fig, axes = plt.subplots(ncols=2)

    for ax, img in zip(axes, [dist, undist]):
        ax.imshow(img)
        ax.set_xticks([]); ax.set_yticks([])
    fig.suptitle(imgPath)

In [ ]:
# Doesn't work for arcane cv2 reasons.
#undistorter.remap(imgPaths[0], optimalMatrix=True, m1type=cv2.CV_32FC1)

# GoPro "WellsFargo"

In [ ]:
ls ~/data2/cameraCalibration/*.MP4

In [ ]:
fpath = '/home/tsbertalan/data2/cameraCalibration/GOPR0019.avi'

In [ ]:
import os, sys
os.listdir(os.path.dirname(fpath))

In [ ]:
import sys
import skvideo.io
reader = skvideo.io.FFmpegReader(fpath)

In [ ]:
frames = [f for f in tqdm.tqdm_notebook(reader.nextFrame(), total=reader.inputframenum)]

In [ ]:
import utils

In [ ]:
from importlib import reload
reload(utils)

In [ ]:
utils.saveVideo(frames[::20][:42], 'trainingFrames.mp4')

In [ ]:
from sklearn.utils import shuffle
undistorter = Undistorter()
undistorter.fit(frames[::20][:42], earlyStoppingRatio=2.5)

In [ ]:
def smooth(x, window_len=4, window='flat'):
    if window == 'flat': #moving average
        w = np.ones(window_len,'d')
    else:
        w = eval('np.'+window+'(window_len)')

    return np.convolve(w/w.sum(), x, mode='valid')

fig, ax = undistorter.reprojectionErrorPlot(label='original')
unsmoothed = undistorter.calibrationErrors
smoothed = smooth(unsmoothed)
x = np.linspace(0, len(unsmoothed), len(smoothed))
ax.plot(
    x,
    smoothed, label='smoothed', #marker='o'
)
ax.legend();
smoothed[-1] / min(unsmoothed)

fig, ax = plt.subplots()
ax.plot(np.array(unsmoothed) / min(unsmoothed), label='unsmoothed')
ax.plot(x, smoothed / min(unsmoothed), label='smoothed');
ax.legend()
ax.set_xlabel('number of images')
ax.set_ylabel('ratio to minimum unsmoothed')

In [ ]:
dist = frames[400]
alpha = 1
udist = undistorter(dist, cropRoi=False, alpha=alpha)
cropped = undistorter(dist, cropRoi=True, alpha=alpha)
fig, axes = plt.subplots(ncols=3)
for ax, frame, title in zip(
    axes, 
    [dist, udist, cropped], 
    ['original', 'undistorted\n' + r'$\alpha=%.2g$' % alpha, 'cropped\n' + r'($\alpha=%.2g$)' % alpha]
):
    ax.imshow(frame)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(title)

In [ ]:
cropped.shape

In [ ]:
# utils.saveVideo(frames, 'original.mp4')

In [ ]:
import gc; gc.collect()

In [ ]:
!rm undist.mp4

In [ ]:
inputs = frames[:200]
vid = utils.saveVideo(
    (undistorter(frame, cropRoi=False, alpha=.5) for frame in inputs),
    'undist.mp4',
    total=len(inputs),
)
# Apparently the video needs time to settle in the filesystem. Dropbox?
import time
time.sleep(4)
# vid

In [ ]:
!rm undist.mp4

In [ ]:
vid